# Time

In [43]:
import pandas as pd

## Database connection

In [44]:
import yaml
from sqlalchemy import create_engine

with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction and transformation

In [45]:
time_dimension = pd.DataFrame(
	{ "date": pd.date_range(start="18/9/2023", end="9/1/2024", freq="min") }
)

In [46]:
time_dimension.head()

,date
0,2023-09-18 00:00:00
1,2023-09-18 00:01:00
2,2023-09-18 00:02:00
3,2023-09-18 00:03:00
4,2023-09-18 00:04:00


In [47]:
time_dimension["year"] = time_dimension["date"].dt.year
time_dimension["month"] = time_dimension["date"].dt.month
time_dimension["day"] = time_dimension["date"].dt.day
time_dimension["day_of_year"] = time_dimension["date"].dt.day_of_year
time_dimension["day_of_month"] = time_dimension["date"].dt.days_in_month
time_dimension["month_str"] = time_dimension["date"].dt.month_name()
time_dimension["day_str"] = time_dimension["date"].dt.day_name()
time_dimension["hour"] = time_dimension["date"].dt.hour
time_dimension["minute"] = time_dimension["date"].dt.minute

time_dimension.head()

,date,year,month,day,day_of_year,day_of_month,month_str,day_str,hour,minute
0,2023-09-18 00:00:00,2023,9,18,261,30,September,Monday,0,0
1,2023-09-18 00:01:00,2023,9,18,261,30,September,Monday,0,1
2,2023-09-18 00:02:00,2023,9,18,261,30,September,Monday,0,2
3,2023-09-18 00:03:00,2023,9,18,261,30,September,Monday,0,3
4,2023-09-18 00:04:00,2023,9,18,261,30,September,Monday,0,4


In [48]:
time_dimension.reset_index(inplace=True)
time_dimension.rename(columns={ "index": "time_id" }, inplace=True)
time_dimension.set_index("time_id", inplace=True)

In [49]:
time_dimension.shape

(502561, 10)

## Load

In [50]:
time_dimension.to_sql(
	"TIME_DIMENSION", OLAP_connection, if_exists="replace", index=True
)

561